SCRAPING - ss 21 
--------------- 
> Chiara Piccolroaz

> University of Konstanz

> Department of Computer and Information Science; Department of political and administration science


IMPORTING PACKAGES
--------------------------

WEbseiten for Scraping Juris : https://www.juris.de/jportal/portal/ and https://rzblx10.uni-regensburg.de/dbinfo/detail.php?bib_id=ubko&colors=&ocolors=&lett=f&tid=0&titel_id=865

WebSeite for Installing Selenium: : https://www.jcchouinard.com/learn-selenium-python-seo-automation/

PS: downloading the ChromeDrive is a bitch

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
import requests, PyPDF2
from io import BytesIO
import pickle 
import sys
from selenium.common.exceptions import NoSuchElementException
import os.path
from selenium.webdriver.common.action_chains import ActionChains

BEFOR STARTING:

Make sure to: (1) be locked in the Uni-WLAN or connected with the Uni-VPN. (2) the packages have been download. (3) your internet connection is really strong and stable, otherwise the programm will probably crush

PROBLEME: 

After circa 30 Minuts you are automatically locked out. Either write a programm to automatically reload the page and keep going from the point it chrushed or do it automatically. For the time being I do it manually


PART 1. Get Links
-------------------------

Open the Web and Select the relevant filers and buttons. Make sure to: run the sleep functions, otherwise the other functions will override. 

In [13]:
# open the web
driverC=webdriver.Chrome(executable_path=r"C:\Program Files\Google\Chrome\Application\chromedriver.exe") 
driverC.get('https://www.juris.de/jportal/?action=JLoginUser&username=HEBISUKN.autologin')
driverC.get('https://www.juris.de/r3/search')
driverC.maximize_window()

# save the coockies
pickle.dump(driverC.get_cookies() , open("QuoraCookies.pkl","wb"))
for cookie in pickle.load(open("QuoraCookies.pkl", "rb")): 
    driverC.add_cookie(cookie)
driverC.implicitly_wait(10) # seconds   
button = driverC.find_element_by_id("categories_hits_Rechtsprechung")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("filter_Sachgebiete")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("filter_Asylis___MILo")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("filter_apply")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("linkWorkId80896")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("menuHitsPerPage")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("menuHitsPerPage_100")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("menuSortingOrder")
button.click()
driverC.implicitly_wait(10) # seconds
button = driverC.find_element_by_id("menuSortingOrder_Datum_aufsteigend")
button.click()
driverC.implicitly_wait(10) # seconds


download the PDFs: 
Collect the link of the pdfs for each of the 20454 elements (the number of elements may have changed in the time). In order to do that use the following function, which enters each of the 20454 element files, get the PDFs-link together with the coockies and save them as PDF. 


In [15]:
# 0. Initialisation

# i refers to the number of ducuments pro page and gives the function the id of the document we are looking for. Put i = j = 0
i = 0 
# j refers to the number of documents in total. Put i = j = 0
j = 0
# p refers to the number of the page. If you want to start by a certain page, change the number of the page but do not forget to move on the right page also in chrome, also page p 
p = 193 # check that p = page number on chrome
# destination directory fo the pdfs
destination_directory = "D:/Urteile"
    
# 1. repeat the function for the number of the documents: 
while j < 14751:
    # get the name
    # create the string, which indicates the element_id on the website, which we want to enter the element file with 
    str1 ='resultlistentry_LI' 
    str2 = "{}".format(i)
    str = str1 + str2
    print(str) 
       
# 3. try to get the pdf and save it.
    try: 
        ## try to get the pdf-link
        #driverC.implicitly_wait(2) # seconds
        pdf_a = driverC.find_element_by_css_selector("a[title='Im neuen Fenster: das folgende PDF-Dokument']")
        driverC.implicitly_wait(2) # seconds
        link = pdf_a.get_attribute('href')    
        driverC.implicitly_wait(2) # seconds
        print("got the PDFs, go one back" + link)
        
        ## try to get the cookies and acess the pdf through the cookies
        cookies = driverC.get_cookies()
        for cookie in cookies:
            if cookie['name'] == 'JSESSIONID':
                session_cookie = cookie['value']
                        
        cookies = {'JSESSIONID': session_cookie}
        r = requests.get(link, cookies=cookies)
        driverC.implicitly_wait(3) # seconds
        
        ## try to save the pdf
        Titl1 ='resultlistentry_LI' 
        Titl2 = "_{}".format(i)
        Titl3 = "_{}".format(p)
        Titl = Titl1 + Titl3 + Titl2  
        OutputFilename = os.path.join(destination_directory, Titl+".pdf")
        f = open(OutputFilename, "wb")
        f.write(r.content)
        f.close()
        print("PDF saved")
        driverC.implicitly_wait(3) # seconds
    
    # 3.1 If it does not work go back and go on with the other pdfs
    except NoSuchElementException as err:
        print("no PDF")  
        ## go back to the main page and go on with the other elements
        Down= driverC.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        driverC.implicitly_wait(5) # seconds 
        print("get the next entry, since no PDF") 
        nextDoc = driverC.find_element_by_id("docnextbuttonbottom")
        nextDoc.click()
        driverC.implicitly_wait(4) # seconds 
        ## increase i to check for the elements pro page; increase j to check for the 19151 elements
        i = i+1
        j = j+1 
        ## if te last element change the page 
        if i == 100: 
            print("i is equal 100 - we got the 100th element of the page -, so change page")  
            p = p+1
            print("NEXT page" , p)
            i= 0
        ## go  on with the other PDFs
        continue
        
    # 3.1 If it does work go back and go on with the other pdfs
    ## go back to the main page and go on with the other elements
    Down= driverC.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    driverC.implicitly_wait(5) # seconds 
    print("get the next entry, since no PDF") 
    nextDoc = driverC.find_element_by_id("docnextbuttonbottom")
    nextDoc.click()
    driverC.implicitly_wait(4) # seconds 
    ## increase i to check for the elements pro page; increase j to check for the 19151 elements
    i = i+1
    j = j+1 
    ## if te last element change the page 
    if i == 100: 
        print("i is equal 100 - we got the 100th element of the page -, so change page")  
        p = p+1
        print("NEXT page" , p) 
        i= 0

resultlistentry_LI0
no PDF
get the next entry, since no PDF
resultlistentry_LI1
no PDF
get the next entry, since no PDF
resultlistentry_LI2
no PDF
get the next entry, since no PDF
resultlistentry_LI3
no PDF
get the next entry, since no PDF
resultlistentry_LI4
no PDF
get the next entry, since no PDF
resultlistentry_LI5
no PDF
get the next entry, since no PDF
resultlistentry_LI6
no PDF
get the next entry, since no PDF
resultlistentry_LI7
no PDF
get the next entry, since no PDF
resultlistentry_LI8
no PDF
get the next entry, since no PDF
resultlistentry_LI9
no PDF
get the next entry, since no PDF
resultlistentry_LI10
no PDF
get the next entry, since no PDF
resultlistentry_LI11
no PDF
get the next entry, since no PDF
resultlistentry_LI12
no PDF
get the next entry, since no PDF
resultlistentry_LI13
no PDF
get the next entry, since no PDF
resultlistentry_LI14
no PDF
get the next entry, since no PDF
resultlistentry_LI15
no PDF
get the next entry, since no PDF
resultlistentry_LI16
no PDF
get th

no PDF
get the next entry, since no PDF
resultlistentry_LI34
no PDF
get the next entry, since no PDF
resultlistentry_LI35
no PDF
get the next entry, since no PDF
resultlistentry_LI36
no PDF
get the next entry, since no PDF
resultlistentry_LI37
no PDF
get the next entry, since no PDF
resultlistentry_LI38
no PDF
get the next entry, since no PDF
resultlistentry_LI39
no PDF
get the next entry, since no PDF
resultlistentry_LI40
no PDF
get the next entry, since no PDF
resultlistentry_LI41
no PDF
get the next entry, since no PDF
resultlistentry_LI42
no PDF
get the next entry, since no PDF
resultlistentry_LI43
no PDF
get the next entry, since no PDF
resultlistentry_LI44
no PDF
get the next entry, since no PDF
resultlistentry_LI45
no PDF
get the next entry, since no PDF
resultlistentry_LI46
no PDF
get the next entry, since no PDF
resultlistentry_LI47
no PDF
get the next entry, since no PDF
resultlistentry_LI48
no PDF
get the next entry, since no PDF
resultlistentry_LI49
no PDF
get the next entr

no PDF
get the next entry, since no PDF
resultlistentry_LI68
no PDF
get the next entry, since no PDF
resultlistentry_LI69
no PDF
get the next entry, since no PDF
resultlistentry_LI70
no PDF
get the next entry, since no PDF
resultlistentry_LI71
no PDF
get the next entry, since no PDF
resultlistentry_LI72
no PDF
get the next entry, since no PDF
resultlistentry_LI73
no PDF
get the next entry, since no PDF
resultlistentry_LI74
no PDF
get the next entry, since no PDF
resultlistentry_LI75
no PDF
get the next entry, since no PDF
resultlistentry_LI76
no PDF
get the next entry, since no PDF
resultlistentry_LI77
no PDF
get the next entry, since no PDF
resultlistentry_LI78
no PDF
get the next entry, since no PDF
resultlistentry_LI79
no PDF
get the next entry, since no PDF
resultlistentry_LI80
no PDF
get the next entry, since no PDF
resultlistentry_LI81
no PDF
get the next entry, since no PDF
resultlistentry_LI82
no PDF
get the next entry, since no PDF
resultlistentry_LI83
no PDF
get the next entr

no PDF
get the next entry, since no PDF
resultlistentry_LI1
no PDF
get the next entry, since no PDF
resultlistentry_LI2
no PDF
get the next entry, since no PDF
resultlistentry_LI3
no PDF
get the next entry, since no PDF
resultlistentry_LI4
no PDF
get the next entry, since no PDF
resultlistentry_LI5
no PDF
get the next entry, since no PDF
resultlistentry_LI6
no PDF
get the next entry, since no PDF
resultlistentry_LI7
no PDF
get the next entry, since no PDF
resultlistentry_LI8
no PDF
get the next entry, since no PDF
resultlistentry_LI9
no PDF
get the next entry, since no PDF
resultlistentry_LI10
no PDF
get the next entry, since no PDF
resultlistentry_LI11
no PDF
get the next entry, since no PDF
resultlistentry_LI12
no PDF
get the next entry, since no PDF
resultlistentry_LI13
no PDF
get the next entry, since no PDF
resultlistentry_LI14
no PDF
get the next entry, since no PDF
resultlistentry_LI15
no PDF
get the next entry, since no PDF
resultlistentry_LI16
no PDF
get the next entry, since 

no PDF
get the next entry, since no PDF
resultlistentry_LI35
no PDF
get the next entry, since no PDF
resultlistentry_LI36
no PDF
get the next entry, since no PDF
resultlistentry_LI37
no PDF
get the next entry, since no PDF
resultlistentry_LI38
no PDF
get the next entry, since no PDF
resultlistentry_LI39
no PDF
get the next entry, since no PDF
resultlistentry_LI40
no PDF
get the next entry, since no PDF
resultlistentry_LI41
no PDF
get the next entry, since no PDF
resultlistentry_LI42
no PDF
get the next entry, since no PDF
resultlistentry_LI43
no PDF
get the next entry, since no PDF
resultlistentry_LI44
no PDF
get the next entry, since no PDF
resultlistentry_LI45
no PDF
get the next entry, since no PDF
resultlistentry_LI46
no PDF
get the next entry, since no PDF
resultlistentry_LI47
no PDF
get the next entry, since no PDF
resultlistentry_LI48
no PDF
get the next entry, since no PDF
resultlistentry_LI49
no PDF
get the next entry, since no PDF
resultlistentry_LI50
no PDF
get the next entr

...

no PDF
get the next entry, since no PDF
resultlistentry_LI1
no PDF
get the next entry, since no PDF
resultlistentry_LI2
no PDF
get the next entry, since no PDF
resultlistentry_LI3
no PDF
get the next entry, since no PDF
resultlistentry_LI4
no PDF
get the next entry, since no PDF
resultlistentry_LI5
no PDF
get the next entry, since no PDF
resultlistentry_LI6
no PDF
get the next entry, since no PDF
resultlistentry_LI7
no PDF
get the next entry, since no PDF
resultlistentry_LI8
no PDF
get the next entry, since no PDF
resultlistentry_LI9
no PDF
get the next entry, since no PDF
resultlistentry_LI10
no PDF
get the next entry, since no PDF
resultlistentry_LI11
no PDF
get the next entry, since no PDF
resultlistentry_LI12
no PDF
get the next entry, since no PDF
resultlistentry_LI13
no PDF
get the next entry, since no PDF
resultlistentry_LI14
no PDF
get the next entry, since no PDF
resultlistentry_LI15
no PDF
get the next entry, since no PDF
resultlistentry_LI16
no PDF
get the next entry, since 

no PDF
get the next entry, since no PDF
resultlistentry_LI35
no PDF
get the next entry, since no PDF
resultlistentry_LI36
no PDF
get the next entry, since no PDF
resultlistentry_LI37
no PDF
get the next entry, since no PDF
resultlistentry_LI38
no PDF
get the next entry, since no PDF
resultlistentry_LI39
no PDF
get the next entry, since no PDF
resultlistentry_LI40
no PDF
get the next entry, since no PDF
resultlistentry_LI41
no PDF
get the next entry, since no PDF
resultlistentry_LI42
no PDF
get the next entry, since no PDF
resultlistentry_LI43
no PDF
get the next entry, since no PDF
resultlistentry_LI44
no PDF
get the next entry, since no PDF
resultlistentry_LI45
no PDF
get the next entry, since no PDF
resultlistentry_LI46
no PDF
get the next entry, since no PDF
resultlistentry_LI47
no PDF
get the next entry, since no PDF
resultlistentry_LI48
no PDF
get the next entry, since no PDF
resultlistentry_LI49
no PDF
get the next entry, since no PDF
resultlistentry_LI50
no PDF
get the next entr

no PDF
get the next entry, since no PDF
resultlistentry_LI69
no PDF
get the next entry, since no PDF
resultlistentry_LI70
no PDF
get the next entry, since no PDF
resultlistentry_LI71
no PDF
get the next entry, since no PDF
resultlistentry_LI72
no PDF
get the next entry, since no PDF
resultlistentry_LI73
no PDF
get the next entry, since no PDF
resultlistentry_LI74
no PDF
get the next entry, since no PDF
resultlistentry_LI75
no PDF
get the next entry, since no PDF
resultlistentry_LI76
no PDF
get the next entry, since no PDF
resultlistentry_LI77
no PDF
get the next entry, since no PDF
resultlistentry_LI78
no PDF
get the next entry, since no PDF
resultlistentry_LI79
no PDF
get the next entry, since no PDF
resultlistentry_LI80
no PDF
get the next entry, since no PDF
resultlistentry_LI81
no PDF
get the next entry, since no PDF
resultlistentry_LI82
no PDF
get the next entry, since no PDF
resultlistentry_LI83
no PDF
get the next entry, since no PDF
resultlistentry_LI84
no PDF
get the next entr

no PDF
get the next entry, since no PDF
resultlistentry_LI1
no PDF
get the next entry, since no PDF
resultlistentry_LI2
no PDF
get the next entry, since no PDF
resultlistentry_LI3
no PDF
get the next entry, since no PDF
resultlistentry_LI4
no PDF
get the next entry, since no PDF
resultlistentry_LI5
no PDF
get the next entry, since no PDF
resultlistentry_LI6
no PDF
get the next entry, since no PDF
resultlistentry_LI7
no PDF
get the next entry, since no PDF
resultlistentry_LI8
no PDF
get the next entry, since no PDF
resultlistentry_LI9
no PDF
get the next entry, since no PDF
resultlistentry_LI10
no PDF
get the next entry, since no PDF
resultlistentry_LI11
no PDF
get the next entry, since no PDF
resultlistentry_LI12
no PDF
get the next entry, since no PDF
resultlistentry_LI13
no PDF
get the next entry, since no PDF
resultlistentry_LI14
no PDF
get the next entry, since no PDF
resultlistentry_LI15
no PDF
get the next entry, since no PDF
resultlistentry_LI16
no PDF
get the next entry, since 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="docnextbuttonbottom"]"}
  (Session info: chrome=92.0.4515.107)
